In [1]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os

from pydub import AudioSegment

In [ ]:
def stereo_to_mono(filepath):
    sound = AudioSegment.from_wav(filepath)
    sound = sound.set_channels(1)
    sound.export(filepath, format = 'wav')

In [29]:
def save_spec(audio_data, samplerate, filepath):
    N_FFT = 1024         
    HOP_SIZE = 1024       
    N_MELS = 128            
    WIN_SIZE = 1024      
    WINDOW_TYPE = 'hann' 
    FEATURE = 'mel'      
    FMIN = 1400 

    S = librosa.feature.melspectrogram(y=audio_data,sr=samplerate,
                                        n_fft=N_FFT,
                                        hop_length=HOP_SIZE, 
                                        n_mels=N_MELS, 
                                        htk=True, 
                                        fmin=FMIN, 
                                        fmax=samplerate/2) 
    #turn off interactive mode, so the plots are not displayed when saved
    plt.ioff()
    plt.figure(figsize=(8, 2))
    librosa.display.specshow(librosa.power_to_db(S**2,ref=np.max), fmin=FMIN,y_axis='linear')
    plt.axis("off")
#     plt.colorbar(format='%+2.0f dB')
    # Color could be useful; debate again.
    plt.gray()
    plt.savefig(filepath, format="png", bbox_inches="tight", pad_inches=0)
    plt.close()
#     plt.show()

In [3]:
%matplotlib inline
def print_plot_play(x, Fs, text=''):
    """1. Prints information about an audio singal, 2. plots the waveform, and 3. Creates player
    
    Notebook: C1/B_PythonAudio.ipynb
    
    Args: 
        x: Input signal
        Fs: Sampling rate of x    
        text: Text to print
    """
    print('%s Fs = %d, x.shape = %s, x.dtype = %s' % (text, Fs, x.shape, x.dtype))
    plt.figure(figsize=(8, 2))
    plt.plot(x, color='gray')
    sd = [np.std([i for i in x])] * len(x)
    plt.plot(sd, color='red')
    plt.xlim([0, x.shape[0]])
    plt.xlabel('Time (samples)')
    plt.ylabel('Amplitude')
    plt.tight_layout()
    plt.show()
    ipd.display(ipd.Audio(data=x, rate=Fs))

In [4]:
def find_calls(data, FS):
    is_new = True
    sd = np.std([i for i in data[:,0] if i > 0])
    return_values = []
    for idx, i in enumerate(data[:,0]):
        if i > sd and is_new:
            onset = idx-1000
            is_new = False
        if not is_new and idx-onset>(FS/4)-1000:
            offset = idx
            is_new = True
            
            return_values.append(data[onset:offset])
            
    return return_values
#             print_plot_play(data[onset:offset], FS)
#             plot_spec(data[onset:offset], FS)
        

In [5]:
def get_start_and_target_directory(start_path):
    os.chdir(start_path)
    os.chdir("..")
    os.chdir("data")
    os.chdir("data_processed")
    target_directory = os.fsencode(str(os.path.abspath(os.curdir)))
    
    os.chdir("..")
    os.chdir("data_raw")
    start_directory = os.fsencode(str(os.path.abspath(os.curdir)))
    
    return start_directory, target_directory

In [6]:
def mk_target_call_dir(target_directory, call_name, filename):
    os.chdir(TARGET_DIRECTORY)
    if not os.path.exists(call_name):
        os.mkdir(call_name)
    os.chdir(call_name)
    
    target_call_directory = os.fsencode(str(os.path.abspath(os.curdir)))
    target_filepath = os.path.join(target_call_directory, os.fsencode(filename))
    
    os.mkdir(target_filepath[:-4])
    os.chdir(target_filepath[:-4])
    
    return os.fsencode(str(os.path.abspath(os.curdir)))

In [7]:
def save_audio_spectro(audio_data, samplerate):
    seperate_calls = find_calls(audio_data, samplerate)
    os.mkdir("audio")
    os.mkdir("spectrogram")
    for index, call_data in enumerate(seperate_calls):
                    
        os.chdir("audio")
        sf.write(str(index)+".wav", call_data, samplerate)
                    
        os.chdir("..")
        os.chdir("spectrogram")
        save_spec(call_data[:,0], 
                    samplerate, 
                    str(index)+".png"
                    )
        os.chdir("..")

In [30]:
names = ["Raincall_huid"]
START_PATH = os.path.abspath(os.getcwd())
i = 0
try:
    
    START_DIRECTORY, TARGET_DIRECTORY = get_start_and_target_directory(START_PATH)
    os.chdir(START_DIRECTORY)
    
    for call_name in names:
        
        os.chdir(call_name)
        file_directory = os.fsencode(os.path.abspath(os.curdir))
        for file in os.listdir(file_directory):
            filename = os.fsdecode(file)
            
            if filename.endswith(".wav"): 
                i += 1                  # filename is already os.fsencode 
                filepath = os.path.join(file_directory, os.fsencode(filename))
                stereo_to_mono(filepath)
                audio_data, samplerate = sf.read(filepath)
                print('Working on file no. ', i)
                target_filepath = mk_target_call_dir(TARGET_DIRECTORY, call_name, filename)
                os.chdir(target_filepath)
                
                sf.write("original_"+filename, audio_data, samplerate)
                
                save_audio_spectro(audio_data, samplerate)
            
            
        os.chdir(START_DIRECTORY)
finally:
    os.chdir(START_PATH)


1
2


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed